In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -d ./dataverse_files /content/drive/MyDrive/dataverse_files.zip

!unzip -d ./dataverse_files/HAM10000_images_part_1 /content/dataverse_files/HAM10000_images_part_1.zip
!unzip -d ./dataverse_files/HAM10000_images_part_2 /content/dataverse_files/HAM10000_images_part_2.zip
!unzip -d ./dataverse_files /content/dataverse_files/ISIC2018_Task3_Test_Images.zip

!unzip -d ./dataverse_files /content/drive/MyDrive/InceptionV3.zip

!unzip -d ./dataverse_files /content/dataverse_files/GAN_Images-20221105T161540Z-001.zip
!unzip -d ./dataverse_files /content/dataverse_files/HAM10000_images_augmented-20221103T160519Z-001.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00295451.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00301381.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00275951.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00250191.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00276751.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00321851.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00293521.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00303391.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00289641.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00277591.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00249841.jpg  
  inflating: ./dataverse_files/HAM10000_images_augmented/ISIC_00271201.jpg  
  inflating: ./dataverse_files/HAM10000_images_au

# 1. Preprocessing

In [3]:
import pandas as pd
import os
import shutil

In [4]:
# move training images into folders

# create train folder
if not os.path.exists('dataverse_files/train'):
    os.mkdir('dataverse_files/train')
    
for img_name in os.listdir('dataverse_files/HAM10000_images_part_1/'):
    src_path = os.path.join('dataverse_files/HAM10000_images_part_1', img_name)
    tgt_path = 'dataverse_files/train'
    shutil.move(src_path, tgt_path)
    
for img_name in os.listdir('dataverse_files/HAM10000_images_part_2/'):
    src_path = os.path.join('dataverse_files/HAM10000_images_part_2', img_name)
    tgt_path = 'dataverse_files/train'
    shutil.move(src_path, tgt_path)

# for img_name in os.listdir('dataverse_files/HAM10000_images_augmented/'):
#     src_path = os.path.join('dataverse_files/HAM10000_images_augmented', img_name)
#     renamed_src_path = os.path.join('dataverse_files/HAM10000_images_augmented', img_name.split('.')[0] + '_aug.jpg')
#     os.rename(src_path, renamed_src_path)
#     tgt_path = 'dataverse_files/train'
#     shutil.move(renamed_src_path, tgt_path)

for img_name in os.listdir('dataverse_files/GAN_Images/'):
    src_path = os.path.join('dataverse_files/GAN_Images', img_name)
    tgt_path = 'dataverse_files/train'
    shutil.move(src_path, tgt_path)

In [6]:
from sklearn.model_selection import train_test_split
train_df = pd.read_csv('dataverse_files/HAM10000_metadata')
# aug_train_df = pd.read_csv('dataverse_files/HAM10000_metadata_augmented.csv').iloc[:, 1:]
# aug_train_df['image_id'] = aug_train_df['image_id'].apply(lambda x: x+'_aug')

gan_df = pd.read_csv('dataverse_files/HAM10000_GAN_data.csv').iloc[:, 1:]

train, test = train_test_split(train_df[['image_id', 'dx']], test_size=0.2)
# train = pd.concat((train, aug_train_df[['image_id', 'dx']]), axis=0)

gan_df['dx'] = gan_df['dx'].map({0: 'bkl', 1: 'nv', 2: 'df', 3: 'mel', 4: 'vasc', 5: 'bcc', 6: 'akiec'})
train = pd.concat((train, gan_df[['image_id', 'dx']]), axis=0)

In [8]:
# convert to numerical representation
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
lb.fit_transform(train_df['dx'])
cls2lbl = {cls: idx for idx, cls in enumerate(lb.classes_)}
lbl2cls = {idx: cls for idx, cls in enumerate(lb.classes_)}
train['label'] = [cls2lbl[cls] for cls in train['dx']]
test['label'] = [cls2lbl[cls] for cls in test['dx']]

# 2. Data loader

In [9]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, RandomAffine, Resize, CenterCrop, Normalize

class MyDataset(Dataset):
    def __init__(self, mode='train'):
        self.meta = train if mode == 'train' else test
        self.root_dir = 'dataverse_files/train'
    def __len__(self):
        return len(self.meta)
    def __getitem__(self, idx):
        if self.meta['image_id'].iloc[idx].startswith('GAN'):
            img_name = self.meta['image_id'].iloc[idx]
        else:
            img_name = self.meta['image_id'].iloc[idx] + '.jpg'
        img_path = os.path.join(self.root_dir, img_name)

        image = Image.open(img_path)
        image = Resize((299, 299))(image)
        image = RandomAffine(degrees=(-10, 10), translate=(0.05, 0.1), scale=(0.9, 1.1))(image)
        image = ToTensor()(image)
        image = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
        
        label = self.meta['label'].iloc[idx]
        return image, label

train_dataset = MyDataset(mode='train')
test_dataset = MyDataset(mode='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

# 3. Models

In [10]:
import torch.nn as nn
from torchvision.models import inception_v3

model = inception_v3(pretrained=False)
# for param in model.parameters():
#     param.requires_grad = False

model.fc = nn.Sequential(nn.Linear(2048, 7))
model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:47: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

# 4. Train

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

n_epochs = 15
lr = 0.001
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
loss_func = torch.nn.CrossEntropyLoss()
best_test_acc = 0

for i in range(n_epochs):
    for idx, (image, label) in enumerate(train_loader):
        image = image.to(device)
        label = label.to(device)
        
        y_pred = model(image)[0]
        loss = loss_func(y_pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (idx + 1) % 10 == 0:
            print(f'Iteration: {i+1}/{n_epochs}, Step: {idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    model.eval()
    with torch.no_grad():
        n_instances = len(test_loader)
        n_correct = 0
        for image, label in test_loader:
            image = image.to(device)
            label = label.to(device)
            
            y_pred = model(image)
            pred = torch.argmax(y_pred, dim=1)
            n_correct += (pred == label).sum().item()
        test_acc = n_correct / n_instances
        print(f'Iteration: {i+1}/{n_epochs}, Test accuracy: {test_acc:.4f}')
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            torch.save(model, 'best_model.pth')
        
    model.train()
    
print(f'Best test accuracy: {best_test_acc:.4f}')

Iteration: 1/15, Step: 10/439, Loss: 1.5433
Iteration: 1/15, Step: 20/439, Loss: 1.6054
Iteration: 1/15, Step: 30/439, Loss: 1.2012
Iteration: 1/15, Step: 40/439, Loss: 1.3254
Iteration: 1/15, Step: 50/439, Loss: 1.2372
Iteration: 1/15, Step: 60/439, Loss: 1.6407
Iteration: 1/15, Step: 70/439, Loss: 1.1545
Iteration: 1/15, Step: 80/439, Loss: 1.7041
Iteration: 1/15, Step: 90/439, Loss: 1.3110
Iteration: 1/15, Step: 100/439, Loss: 1.0668
Iteration: 1/15, Step: 110/439, Loss: 1.0945
Iteration: 1/15, Step: 120/439, Loss: 1.4124
Iteration: 1/15, Step: 130/439, Loss: 1.2590
Iteration: 1/15, Step: 140/439, Loss: 1.1450
Iteration: 1/15, Step: 150/439, Loss: 1.3972
Iteration: 1/15, Step: 160/439, Loss: 0.8953
Iteration: 1/15, Step: 170/439, Loss: 1.6367
Iteration: 1/15, Step: 180/439, Loss: 1.1404
Iteration: 1/15, Step: 190/439, Loss: 1.2625
Iteration: 1/15, Step: 200/439, Loss: 1.2789
Iteration: 1/15, Step: 210/439, Loss: 1.1281
Iteration: 1/15, Step: 220/439, Loss: 1.1435
Iteration: 1/15, St

In [12]:
!cp /content/best_model.pth /content/drive/MyDrive

# 5. Evaluate

In [13]:
best_model = torch.load('best_model.pth')
best_model.eval()
with torch.no_grad():
    y_preds = []
    y_trues = []
    for image, label in test_loader:
        image = image.to(device)
        label = label.to(device)
        
        y_pred = best_model(image)
        pred = torch.argmax(y_pred, dim=1)

        y_trues.append(label.cpu().numpy())
        y_preds.append(pred.cpu().numpy())


In [ ]:
import numpy as np
from sklearn.metrics import recall_score

y_preds = np.concatenate(y_preds)
y_trues = np.concatenate(y_trues)

scores = recall_score(y_trues, y_preds, average=None)
scores

In [16]:
from sklearn.metrics import accuracy_score

test_acc = accuracy_score(y_trues, y_preds)
test_acc

0.7678482276585122